In [ ]:
import os
import sys
import glob
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import zipfile
import cv2
import torch
import torchvision.transforms as T
import torchvision.models as models
import torchvision

In [ ]:
def find_test_folder(possible_names=None):
    # common Kaggle pattern: /kaggle/input/<dataset-name>/
    if possible_names is None:
        possible_names = [
            "recodai-luc-scientific-image-forgery-detection",
            "recodai_luc_scientific_image_forgery_detection",
            "recodai-luc-scientific-image-forgery",
            "test_images", "test", "test_images_png"
        ]
    # check /kaggle/input
    kaggle_input = Path("/kaggle/input")
    candidates = []
    if kaggle_input.exists():
        for d in kaggle_input.glob("*"):
            # check for a test or images subfolder
            for name in possible_names:
                p = d / name
                if p.exists():
                    candidates.append(str(p))
            # also check likely structure: dataset/test or dataset/test_images
            for p in (d / "test", d / "test_images", d / "images"):
                if p.exists():
                    candidates.append(str(p))
    # fallback: current directory
    for name in possible_names:
        if Path(name).exists():
            candidates.append(name)
    # choose first candidate
    if candidates:
        return candidates[0]
    # if nothing found, instruct user
    raise FileNotFoundError("Could not find test image folder. Please attach the competition dataset or set 'TEST_FOLDER' manually.")

# Example
try:
    TEST_FOLDER = find_test_folder()
    print("Auto-detected test folder:", TEST_FOLDER)
except FileNotFoundError as e:
    print(e)
    # If running locally, set TEST_FOLDER manually:
    # TEST_FOLDER = "/path/to/test/images"
    TEST_FOLDER = None

In [ ]:
def list_image_files(test_folder):
    exts = ("*.png", "*.jpg", "*.jpeg", "*.tif", "*.tiff", "*.bmp")
    files = []
    for e in exts:
        files.extend(sorted(Path(test_folder).glob(e)))
    return [str(p) for p in files]

if TEST_FOLDER:
    test_files = list_image_files(TEST_FOLDER)
    print(f"Found {len(test_files)} test images (showing up to 5):")
    for p in test_files[:5]:
        print(" ", p)
else:
    test_files = []

In [ ]:
# Cell 4 - RLE encoding/decoding utilities (standard Kaggle format)
# RLE encodes mask in column-major order, 1-indexed positions (common in Kaggle competitions).
def rle_encode(mask):
    '''
    mask: 2D numpy array {0,1}, 1 - mask foreground
    returns run-length as string
    '''
    pixels = mask.flatten(order='F')  # Fortran order (column-major)
    # add sentinel
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] = runs[1::2] - runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle

def rle_decode(rle, shape):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, ln in zip(starts, lengths):
        img[lo:lo+ln] = 1
    return img.reshape((shape[1], shape[0]), order='F').T  # careful reshape back

In [ ]:
# Cell 5 - Baseline predictor (very fast, deterministic)
# Strategy: convert to grayscale, use Otsu threshold + morphological opening to remove noise.
def baseline_predict(img_path, resize_to=None):
    """
    Input: image path
    Output: binary mask (numpy uint8) same size as input image (0/1)
    """
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Could not read {img_path}")
    orig_h, orig_w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # optional resizing for speed
    scale = 1.0
    if resize_to is not None:
        h = int((resize_to * orig_h) ** 0.5) # dummy, but we won't use this; keep as placeholder
    # Otsu threshold
    _, thr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # morphological operations to clean
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    clean = cv2.morphologyEx(thr, cv2.MORPH_OPEN, kernel, iterations=1)
    # optionally apply Canny edges and combine (helps find copy-move edges)
    edges = cv2.Canny(gray, 50, 150)
    combined = cv2.bitwise_or(clean, edges)
    # final binary mask
    mask = (combined > 0).astype(np.uint8)
    # Fill small holes and remove tiny objects
    # remove small connected components
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    out_mask = np.zeros_like(mask)
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= 50:  # tuneable threshold
            out_mask[labels == i] = 1
    return out_mask

# Quick test visual (only if there are images)
if test_files:
    sample = test_files[0]
    print("Sample:", sample)
    m = baseline_predict(sample)
    print("Mask shape:", m.shape, "unique:", np.unique(m))
    # visualize
    img = cv2.imread(sample)[:,:,::-1]
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1); plt.imshow(img); plt.axis('off'); plt.title('Image')
    plt.subplot(1,2,2); plt.imshow(m, cmap='gray'); plt.axis('off'); plt.title('Baseline mask')
    plt.show()

In [ ]:
# Cell 6 - Optional: simple deep model inference (FCN) for better baseline
# NOTE: This uses torchvision's pretrained segmentation model (FCN/DeepLab). It may require GPU and will not be perfect for this domain.
def fcn_predict(img_path, device='cuda'):
    model = models.segmentation.fcn_resnet50(pretrained=True).eval().to(device)
    # remove after first call or move outside for speed in real notebook
    transform = T.Compose([T.ToTensor(),
                           T.Normalize(mean=[0.485, 0.456, 0.406],
                                       std =[0.229, 0.224, 0.225])])
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)['out'][0]  # 21 classes for fcn pretrained on COCO/Pascal?
        # take max across classes and threshold
        scores = torch.softmax(out, dim=0)
        # simple heuristic: foreground = any class except background (class 0 often)
        fg = scores[1:].sum(dim=0).cpu().numpy()
        mask = (fg > 0.3).astype(np.uint8)  # threshold tunable
    # resize to original if needed (it's same size)
    return mask

# Warning: downloading model weights may take time and GPU is beneficial.

In [ ]:
# Cell 7 - Create predictions and produce submission files
OUTPUT_DIR = Path("submission_output")
OUTPUT_DIR.mkdir(exist_ok=True)

def make_submission_rle(test_files, predictor_func, output_csv="submission.csv", verbose=True):
    rows = []
    for p in tqdm(test_files, desc="Predicting"):
        fname = Path(p).name
        mask = predictor_func(p)
        # ensure mask is binary 0/1 and same size as image
        if mask.dtype != np.uint8:
            mask = mask.astype(np.uint8)
        # encode to rle (if competition expects column-major 1-indexed)
        rle = rle_encode(mask)
        rows.append({"image_id": fname, "EncodedPixels": rle})
    df = pd.DataFrame(rows)
    df.to_csv(OUTPUT_DIR / output_csv, index=False)
    if verbose:
        print("Wrote", OUTPUT_DIR / output_csv)
    return df

def make_submission_pngs(test_files, predictor_func, zip_name="masks.zip", out_mask_size=None):
    mask_folder = OUTPUT_DIR / "masks"
    mask_folder.mkdir(exist_ok=True)
    for p in tqdm(test_files, desc="Predicting PNGs"):
        fname = Path(p).stem
        mask = predictor_func(p)
        # ensure 0/255 PNG
        mask_img = (mask * 255).astype(np.uint8)
        pil = Image.fromarray(mask_img)
        # optional: resize to original dims? already same
        pil.save(mask_folder / f"{fname}.png")
    # zip them
    zip_path = OUTPUT_DIR / zip_name
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        for f in sorted(mask_folder.glob("*.png")):
            zf.write(f, arcname=f.name)
    print("Saved masks zip:", zip_path)
    return zip_path

In [ ]:
# Cell 8 - Run baseline to create RLE CSV and masks.zip
if not test_files:
    print("No test files detected. Set TEST_FOLDER or upload test images.")
else:
    # Baseline RLE
    df_rle = make_submission_rle(test_files, baseline_predict, output_csv="submission_rle_baseline.csv")
    # Baseline PNGs ZIP
    zip_path = make_submission_pngs(test_files, baseline_predict, zip_name="masks_baseline.zip")
    display(df_rle.head())

In [ ]:
# Cell 9 - (Optional) Run FCN model for better masks (slow)
# Uncomment to use FCN. On Kaggle set accelerator to GPU.
# WARNING: This will load pretrained weights and use GPU if available.
"""
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def fcn_wrapper(p):
    return fcn_predict(p, device=device)

df_rle_fcn = make_submission_rle(test_files, fcn_wrapper, output_csv="submission_rle_fcn.csv")
zip_path_fcn = make_submission_pngs(test_files, fcn_wrapper, zip_name="masks_fcn.zip")
"""

In [ ]:
# Cell 10 - How to submit
print("Files generated in:", OUTPUT_DIR.resolve())
print("- submission_rle_baseline.csv  (RLE CSV format)")
print("- masks_baseline.zip           (ZIP of PNG masks)")
print()
print("To submit on Kaggle: go to the competition -> Submit Predictions -> choose either CSV or ZIP (depending on competition rules).")